# ClimateQ&A
---
Goal of the notebook: Recommended graphs functionality

Inputs of the notebook:

Output of the notebook:


Takeaways:

Questions, thoughts and remarks:
- What do I put for query instruction ?
  - Default is "Represent this sentence for searching relevant passages:"
  - embedding_function = get_embeddings_function(query_instruction="")

## Dependencies and path
Adjust the argument in `sys.path.append` to align with your specific requirements.

In [3]:
import pandas as pd 
import numpy as np
import os

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/home/dora/climate-question-answering")

from dotenv import load_dotenv
load_dotenv()

True

## 1. Import objects
### 1.1 LLM

In [4]:
from climateqa.engine.llm import get_llm
llm = get_llm(provider="openai")

### 1.2 Reranker

In [5]:
from climateqa.engine.reranker import get_reranker

reranker = get_reranker("jina")

/home/dora/anaconda3/envs/climateqa_huggingface/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dora/anaconda3/envs/climateqa_huggingface/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 1.3 Vectorstore
#### 1.3.1 IPCC data

In [6]:
from climateqa.engine.vectorstore import get_pinecone_vectorstore
from climateqa.engine.embeddings import get_embeddings_function

embeddings_function = get_embeddings_function()
vectorstore = get_pinecone_vectorstore(embeddings_function)

Loading embeddings model:  BAAI/bge-base-en-v1.5


/home/dora/anaconda3/envs/climateqa_huggingface/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/dora/anaconda3/envs/climateqa_huggingface/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


#### 1.3.2 IEA data

In [46]:
from langchain_community.document_loaders import DataFrameLoader
from langchain_chroma import Chroma

df_iea = pd.read_csv("/home/dora/climate-question-answering/data/charts_iea.csv")
df_iea = df_iea.rename(columns={'url': 'returned_content'})
df_iea["doc_id"] = "iea_" + df_iea.index.astype(str)
df_iea.head()

,title,returned_content,sources,notes,appears_in,appears_in_url,doc_id
0,Capital requirements for mining to meet demand...,https://www.iea.org/data-and-statistics/charts...,IEA analysis based on data from S&P Global and...,Capital requirements are calculated based on c...,Global Critical Minerals Outlook 2024,https://www.iea.org/reports/global-critical-mi...,iea_0
1,"IEA energy transition mineral price index, Jan...",https://www.iea.org/data-and-statistics/charts...,IEA analysis based on Bloomberg and S&P Global.,IEA energy transition minerals price index is ...,Global Critical Minerals Outlook 2024,https://www.iea.org/reports/global-critical-mi...,iea_1
2,Price developments of minerals and metals by c...,https://www.iea.org/data-and-statistics/charts...,IEA analysis based on Bloomberg and S&P Global.,"Base metals include iron, aluminium, zinc and ...",Global Critical Minerals Outlook 2024,https://www.iea.org/reports/global-critical-mi...,iea_2
3,Capital expenditure on nonferrous metal produc...,https://www.iea.org/data-and-statistics/charts...,IEA analysis based on company annual reports a...,"For diversified majors, capex on the productio...",Global Critical Minerals Outlook 2024,https://www.iea.org/reports/global-critical-mi...,iea_3
4,"Selected environmental, social and governance ...",https://www.iea.org/data-and-statistics/charts...,IEA analysis based on the latest sustainabilit...,GHG= greenhouse gas. Aggregated data for 25 ma...,Global Critical Minerals Outlook 2024,https://www.iea.org/reports/global-critical-mi...,iea_4


In [51]:
# Load csv file of charts
loader = DataFrameLoader(df_iea, page_content_column='title')
documents_iea = loader.load()

In [55]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("recommended_graphs")
collection.add(ids=df_iea.doc_id.tolist(), documents=documents_iea)

/home/dora/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:15<00:00, 5.46MiB/s]


TypeError: TextInputSequence must be str

In [15]:
# DO NOT RUN AGAIN (persisted)
vectorstore = Chroma.from_documents(documents, embeddings_function, persist_directory="/home/dora/climate-question-answering/data/vectorstore")

OperationalError: attempt to write a readonly database

In [16]:
from climateqa.engine.vectorstore import get_chroma_vectorstore

iea_vectorstore = get_chroma_vectorstore(embedding_function=embeddings_function)

In [17]:
iea_vectorstore.similarity_search("What is hydrogen import evolutions?")

[Document(page_content='Low-emissions hydrogen imports into Northwest Europe by status, based on announced projects, 2030-2040', metadata={'appears_in': 'Regional cooperation key to tap into the North Sea’s hydrogen potential', 'appears_in_url': 'https://www.iea.org/commentaries/regional-cooperation-key-to-tap-into-the-north-seas-hydrogen-potential', 'sources': 'IEA Global Hydrogen Review 2023, https://www.iea.org/reports/global-hydrogen-review-2023.', 'url': 'https://www.iea.org/data-and-statistics/charts/low-emissions-hydrogen-imports-into-northwest-europe-by-status-based-on-announced-projects-2030-2040'}),
 Document(page_content='Reduction in global hydrogen supply by process route in the Reduced Innovation Case, relative to the Sustainable Development Scenario, 2030-2040', metadata={'appears_in': 'Clean Energy Innovation', 'appears_in_url': 'https://www.iea.org/reports/clean-energy-innovation', 'notes': 'H2 = hydrogen; CCS = carbon capture and storage.', 'url': 'https://www.iea.org

#### 1.3.3 OWID data

In [49]:
df_owid = pd.read_csv("/home/dora/climate-question-answering/data/charts_owid.csv")

# rename column 'embedding' to 'returned_content'
df_owid = df_owid.rename(columns={'embedding': 'returned_content'})
df_owid.head()

df_owid["doc_id"] = "owid_" + df_owid.index.astype(str)
df_owid.head()

,category,title,url,returned_content,subtitle,doc_id
0,Access to Energy,Number of people with and without access to cl...,https://ourworldindata.org/grapher/number-with...,"<iframe src=""https://ourworldindata.org/graphe...",Clean cooking fuels and technologies represent...,owid_0
1,Access to Energy,Number of people without access to clean fuels...,https://ourworldindata.org/grapher/number-with...,"<iframe src=""https://ourworldindata.org/graphe...",Clean cooking fuels and technologies represent...,owid_1
2,Access to Energy,"People without clean fuels for cooking, by wor...",https://ourworldindata.org/grapher/people-with...,"<iframe src=""https://ourworldindata.org/graphe...",Data source: World Bank,owid_2
3,Access to Energy,Share of the population without access to clea...,https://ourworldindata.org/grapher/share-of-th...,"<iframe src=""https://ourworldindata.org/graphe...",Access to clean fuels or technologies such as ...,owid_3
4,Access to Energy,Share with access to electricity vs. per capit...,https://ourworldindata.org/grapher/share-with-...,"<iframe src=""https://ourworldindata.org/graphe...",Having access to electricity is defined in int...,owid_4


In [50]:
loader = DataFrameLoader(df_owid, page_content_column='title')
documents_owid = loader.load()